# Example
[source](https://computationalmindset.com/en/neural-networks/experiments-with-neural-odes-in-python-with-tensorflowdiffeq.html)

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras.optimizers as tfko
from tfdiffeq import odeint

2023-03-21 00:13:13.064118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# !pip install --upgrade git+https://github.com/titu1994/tfdiffeq.git

In [3]:
def LWR_model_tf(t, X, args):

    N, v0, L, flag = \
    args[0], args[1], args[2], args[3]
    
    # W function
    match flag:
        case "Lin":
            W_lin = lambda z: v0*(1-1/z)
            W = W_lin
        case "Log":
            W_log = lambda z: v0*tf.math.log(z)
            W = W_log
        case _:
            return f"No match for {flag}, you can only choose between \"Lin\" and \"Log\""

    # ode sys
    d_x = tf.Variable(tf.zeros(N, tf.float64))
    
    for i in range(0,N-1):
        tmp = (X[i+1] - X[i])/L
        d_x = d_x[i].assign(W(tmp))

    d_x = d_x[N-1].assign(v0)
        
    return tf.stack(d_x)

In [4]:
def net():
    return odeint(lambda ts, x0: LWR_model_tf(ts,x0,args), X_init, t_space_tensor)

def loss_func(num_sol):
    return tf.reduce_sum(tf.square(dataset_outs - num_sol))

In [5]:
#true_params = [1.11, 2.43, -3.66, 1.37, 2.89, -1.97, 4.58, 2.86]

t_begin=0.
t_end=1.5
t_nsamples=150
t_space = np.linspace(t_begin, t_end, t_nsamples)
t_space_tensor = tf.constant(t_space)

N = 3                                       # numbers of vehicles
v0, L = 2, 3                                # control param
flag = "Lin"
X0 = sorted(np.random.uniform(size=N)*100)  # initial condition
X_init = tf.convert_to_tensor(X0, dtype=t_space_tensor.dtype)

names = ['v0','L']
arguments = [v0,L]

args = [tf.Variable(initial_value=N, name='N', dtype="int32")]
tmp = [tf.Variable(initial_value=arguments[i], name=names[i], trainable=True,
                    dtype=t_space_tensor.dtype) for i in range(0,len(names))]
args.append(tmp[0])
args.append(tmp[1])
args.append(tf.Variable(initial_value=flag, name='flag'))

2023-03-21 00:13:14.749973: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 00:13:14.751060: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
args

[<tf.Variable 'N:0' shape=() dtype=int32, numpy=3>,
 <tf.Variable 'v0:0' shape=() dtype=float64, numpy=2.0>,
 <tf.Variable 'L:0' shape=() dtype=float64, numpy=3.0>,
 <tf.Variable 'flag:0' shape=() dtype=string, numpy=b'Lin'>]

In [7]:
dataset_outs = net()

In [8]:
learning_rate = 0.05
epochs = 20
optimizer = tfko.Adam(learning_rate=learning_rate)

In [9]:
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        num_sol = net()
        loss_value = loss_func(num_sol)
        print("Epoch:", epoch, " loss:", loss_value.numpy())

print("Learned parameters:", [args[i].numpy() for i in range(0, 4)])

Epoch: 0  loss: 0.0
Epoch: 1  loss: 0.0
Epoch: 2  loss: 0.0
Epoch: 3  loss: 0.0
Epoch: 4  loss: 0.0
Epoch: 5  loss: 0.0
Epoch: 6  loss: 0.0
Epoch: 7  loss: 0.0
Epoch: 8  loss: 0.0
Epoch: 9  loss: 0.0
Epoch: 10  loss: 0.0
Epoch: 11  loss: 0.0
Epoch: 12  loss: 0.0
Epoch: 13  loss: 0.0
Epoch: 14  loss: 0.0
Epoch: 15  loss: 0.0
Epoch: 16  loss: 0.0
Epoch: 17  loss: 0.0
Epoch: 18  loss: 0.0
Epoch: 19  loss: 0.0
Learned parameters: [3, 2.0, 3.0, b'Lin']


## Compare NN solution with the exact one

In [10]:
num_sol = net()
x_num_sol = num_sol[:, 0].numpy()
y_num_sol = num_sol[:, 1].numpy()

x_an_sol = an_sol_x(t_space)
y_an_sol = an_sol_y(t_space)

NameError: name 'an_sol_x' is not defined

In [ ]:
plt.figure()
plt.plot(t_space, x_an_sol,'--', linewidth=2, label='analytical x')
plt.plot(t_space, y_an_sol,'--', linewidth=2, label='analytical y')
plt.plot(t_space, x_num_sol, linewidth=1, label='numerical x')
plt.plot(t_space, y_num_sol, linewidth=1, label='numerical y')
plt.title('Neural ODEs to fit params')
plt.xlabel('t')
plt.legend()
plt.show()